# MASTER CONTROLLER: ROICaT

In [ ]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [ ]:
import os
from pathlib import Path
import getpass
import gc
import time
import re

import natsort
import paramiko
from tqdm import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2
import bnpm
from bnpm import server

In [ ]:
# paramiko.util.log_to_file("paramiko.log")

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [ ]:
# ## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
# dir_data_local = '/media/rich/bigSSD/tmp_data/'
# dir_data_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/data/test'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

# print(f'contents of    {dir_data_local}    will be copied to    {dir_data_MICROSCOPE}')

In [ ]:
# remote_host_transfer = "transfer.rc.hms.harvard.edu"
# remote_host_compute = "o2.hms.harvard.edu"
# username = input('Username: ')
# pw = util.pw_encode(getpass.getpass(prompt='Password: '))

## Prepare `sftp` and `ssh` objects

In [ ]:
# sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
# sftp.connect(username=username, password=util.pw_decode(pw))

In [ ]:
# # sftp.mkdir_safe(dir_data_remote)
# sftp.put_dir(dir_data_local, dir_data_MICROSCOPE)

------------
# ==== Start ====

## Prepare username + password

In [ ]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = server.pw_encode(getpass.getpass(prompt='Password: '))

## Prepare `sftp` and `ssh` objects

In [ ]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

In [ ]:
sftp_t = server.sftp_interface(ssh_t.client)

# Download data to local

In [ ]:
sftp_t = server.sftp_interface(ssh_client=ssh_t)

In [ ]:
paths_stat_raw = sftp_t.search_recursive(
#     path='/n/files/Neurobio/MICROSCOPE/Rich/data/res2p/old_data/scanimage data/round 5 experiments/mouse 2_6',
    path='/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L',
    search_pattern_re='stat.npy',
    max_depth=10,
);

In [ ]:
def str_contains(s, t):
    import numpy as np
    return np.where([s_i.find(t) >= 0 for s_i in s])[0][0]

In [ ]:
paths_stat = natsort.natsorted([p for p in paths_stat_raw if '/exp/' in p])
display(paths_stat)

dates = natsort.natsorted([re.search('\d{8}', p).group() for p in paths_stat])
display(dates)

paths_stat = {date: paths_stat[str_contains(paths_stat, date)] for date in dates}
display(paths_stat)

In [ ]:
paths_ops = {date: str(Path(path).parent / 'ops.npy') for date, path in paths_stat.items()}

In [ ]:
assert all([sftp_t.exists(path) for path in paths_stat.values()])
assert all([sftp_t.exists(path) for path in paths_ops.values()])

In [ ]:
dir_save_local = r'/media/rich/bigSSD/analysis_data/statFiles_mouse_1511L'

In [ ]:
[sftp_t.get(
    remotepath=path,
    localpath=str(Path(dir_save_local) / date / Path(path).name),
    prog_bar=False,
) for date,path in tqdm(paths_stat.items())];

In [ ]:
[sftp_t.get(
    remotepath=path,
    localpath=str(Path(dir_save_local) / date / Path(path).name),
    prog_bar=False,
) for date,path in tqdm(paths_ops.items())];

# Run remotely

In [ ]:
## initialize ssh_compute
ssh_c = util.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

## Pull/update repo

In [ ]:
ssh_c.send('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/ROICaT')
ssh_c.send_receive('git pull')

ssh_c.expect(str_success=f'[{username}', total_timeout=2);

## Activate environment

In [ ]:
ssh_c.send_receive('source activate ROICaT')

ssh_c.expect(str_success=f'[{username}', total_timeout=2);

## Prepare directories

In [ ]:
import numpy as np

dir_outer = 'round_5_experiments'
mouse_name = 'mouse_2_6'
plane_name = 'exp/suite2p/plane0'
# date = ''

name_slurm = mouse_name + '_' + plane_name[-1:]


dir_data_remote = (Path('/n/data1/hms/neurobio/sabatini/rich/data/res2p/') / dir_outer / mouse_name).as_posix()

dir_saveOutputs = (Path('/n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking') / mouse_name).as_posix()

dir_ROInet_networkFiles = Path('/n/data1/hms/neurobio/sabatini/rich/analysis/ROI_tracking/ROInet_temp_networkFiles').as_posix()


name_job = 'jobNum_'


path_dispatcher_local = str(Path(r'/media/rich/Home_Linux_partition/github_repos/ROICaT/scripts/slurm_dispatching/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
path_dispatcher_remote = str((Path(dir_saveOutputs) / 'dispatcher.py'))  ## path to where you want the dispatcher.py file to be copied onto the server
path_script_remote = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/ROICaT/scripts/slurm_dispatching/tracking_script.py'))  ## path to a copy of the remote script.py

In [ ]:
display(f'{dir_data_remote = }')
display(f'{dir_saveOutputs = }')
display(f'{dir_ROInet_networkFiles = }')
display(f'{path_dispatcher_local = }')
display(f'{path_dispatcher_remote = }')
display(f'{path_script_remote = }')

## Prepare commands

In [ ]:
## Expectation for dispatch args: 
# path_selfScript = args[0] = path_dispatcher_remote
# dir_save = args[1] = dir_S2pOutput_remote
# path_script = args[2] = path_s2pScript_remote
# name_job = args[3] = name_job
# dir_fastDisk = args[4] = dir_fastDisk_remote
# name_slurm = args[5] = name_slurm
# dir_data = args[6] = dir_data_remote


prompt_snip = f'[{username}'

commands = {
    'make_dir': f"mkdir -p {dir_data_remote}",
#     'copy_s2p': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_data_remote} {plane_name} {dir_ROInet_networkFiles}"  ## dispatcher expecting these args as inputs
}
display(commands)

## Upload `dispatcher.py` file

In [ ]:
sftp.mkdir_p(Path(path_dispatcher_remote).parent.as_posix())
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Dispatch `remote_run_s2p.py`

In [ ]:
ssh_c.send(commands['dispatch']);

ssh_c.expect(str_success=prompt_snip);

### check on job

In [ ]:
ssh_c.send(cmd=f'squeue -u {username} --Format=jobid,name,partition,state,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres');
test = ssh_c.expect(str_success=f"[{username}", total_timeout=10);

In [ ]:
## I think this code block caused an out of memory error on a few of my jobs... not sure why

# ssh_c.send(cmd=f"cat {(Path(dir_S2pOutput_remote) / (name_job+'0') / 'print*').as_posix()}")
# ssh_c.expect(str_success=f"[{username}", total_timeout=1);

In [ ]:
ssh_c.send(cmd=f"cat {(Path(dir_saveOutputs) / (name_job+'0') / 'print*').as_posix()}")
ssh_c.expect(str_success=f"RUN COMPLETE", recv_timeout=0.3, total_timeout=60*60*10, sleep_time=0.1, verbose=False);

print(f'RUN COMPLETE!!!     {time.ctime()}')

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs back from `data1` to `MICROSCOPE`

## Prepare username + password

In [ ]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

## Prepare `ssh` object

In [ ]:
## initialize ssh_transfer
ssh_t = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,
)

In [ ]:
dir_S2pOutput_remote = dir_S2pOutput_remote  ## from above
dir_s2pOutput_MICROSCOPE = (Path(dir_data_MICROSCOPE) / 'suite2p_o2_output').as_posix()  ## from above
print(f'contents of    {dir_S2pOutput_remote}    will be copied to    {dir_s2pOutput_MICROSCOPE}')

In [ ]:
commands = {
    'make_dir': f"mkdir -p {dir_s2pOutput_MICROSCOPE}",
    'copy_s2p': f"cp -r {(Path(dir_S2pOutput_remote)).as_posix()}/. {dir_s2pOutput_MICROSCOPE}",
}
display(commands)

In [ ]:
ssh_t.send_receive(commands['make_dir']);

In [ ]:
ssh_t.send(commands['copy_s2p'])
ssh_t.expect(str_success=f'[{username}', total_timeout=120, verbose=True);

## Clean up

In [ ]:
## delete password
del pw
gc.collect()
    
## close ssh
ssh_t.close()
del ssh_t
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs from `MICROSCOPE` to local machine

In [ ]:
dir_s2pOutput_MICROSCOPE = dir_s2pOutput_MICROSCOPE  ## from above
dir_s2pOutput_local = str(Path(dir_data_local).resolve() / 'suite2p_o2_output')
print(f'contents of    {dir_s2pOutput_MICROSCOPE}    will be copied to    {dir_s2pOutput_local}')

In [ ]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

## Prepare `sftp` and `ssh` objects

In [ ]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [ ]:
sftp.get_dir(dir_s2pOutput_MICROSCOPE, dir_s2pOutput_local)

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
sftp.close()
del sftp

In [ ]:
ssh_t.close()
ssh_c.close()